In [47]:
import pandas as pd
from pandasql import sqldf
import numpy as np

In [48]:
all_user_ids = np.arange(1, 1001)
all_product_ids = np.arange(1, 101)
n = 10000
user_ids    = np.random.choice(all_user_ids, n)
product_ids = np.random.choice(all_product_ids, n)

In [49]:
start_date = pd.to_datetime('2022-01-01')

In [50]:
times = pd.date_range(start_date, periods=n, freq='1min')

In [51]:
user_actions = pd.DataFrame({'user_id': user_ids, 
                             'product_id': product_ids, 
                             'time': times})

In [52]:
user_actions['action'] = 'view'

In [53]:
def generate_funel_actions(user_id, product_id, time):
    to_cart = 0.2
    to_purchase = 0.4
    
    df = pd.DataFrame()
    
    if np.random.binomial(1, to_cart, 1)[0]:
        df = pd.DataFrame({
                           'user_id'   : user_id, 
                           'product_id': product_id, 
                           'time'      : time + pd.Timedelta(5, unit='s'), 
                           'action'    : 'add to cart'}, index=[0])
        
        if np.random.binomial(1, to_purchase, 1)[0]:
            df_purchase = pd.DataFrame({
                           'user_id'   : user_id, 
                           'product_id': product_id, 
                           'time'      : time + pd.Timedelta(10, unit='s'), 
                           'action'    : 'purchase'}, index=[0])
    
            df = df.append(df_purchase)
    return df

In [54]:
to_cart_df = pd.DataFrame()

In [55]:
for index, row in user_actions.iterrows():
    user_df = generate_funel_actions(row['user_id'], row['product_id'], row['time'])
    to_cart_df = to_cart_df.append(user_df)

In [56]:
co_cart_df['action'].unique()

array(['add to cart', 'purchase'], dtype=object)

In [57]:
user_actions = user_actions.append(to_cart_df)
user_actions = user_actions.sort_values('time')
user_actions['date'] = user_actions.time.dt.date

In [58]:
q = """SELECT date, 
              views, 
              carts, 
              purchases, 
              100 * purchases / views as purchase_percantage 
       FROM (
           SELECT date, 
            count(case when action = 'view'        then 1 else NULL end) as views, 
            count(case when action = 'add to cart' then 1 else NULL end) as carts, 
            count(case when action = 'purchase'    then 1 else NULL end) as purchases 
           FROM user_actions 
           GROUP BY date);"""

In [59]:
sqldf(q)

TypeError: __init__() got multiple values for argument 'schema'